In [ ]:
# =====================================================
# 🔹 FMNAV Multi-Decoder & Visualizer (Colab)
# =====================================================
import struct, os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
from google.colab import drive
drive.mount('/content/drive')

# 📁 Path to your FMNAV file in Drive
input_file = "/content/drive/MyDrive/output/20250907-150827_Rtk.fmnav"
MAX_RECORDS = 50000

if not os.path.exists(input_file):
    raise FileNotFoundError(f"❌ File not found: {input_file}")

# =====================================================
# Candidate struct layouts
# =====================================================
candidates = {
    "A_Qiii" :  ("<Qiii",  ["timestamp","x_i32","y_i32","z_i32"]),
    "B_QiiiH":  ("<QiiiH", ["timestamp","x_i32","y_i32","z_i32","quality_u16"]),
    "C_QIII" :  ("<QIII",  ["timestamp","x_u32","y_u32","z_u32"]),
    "D_QhhhH":  ("<QhhhH", ["timestamp","x_i16","y_i16","z_i16","quality_u16"]),
    "E_Qfff" :  ("<Qfff",  ["timestamp","x_f","y_f","z_f"]),
}

# =====================================================
# Decode helper
# =====================================================
def decode_file(fmt, max_records=MAX_RECORDS):
    rec_size = struct.calcsize(fmt)
    data = []
    with open(input_file, "rb") as f:
        for _ in range(max_records):
            b = f.read(rec_size)
            if not b or len(b) < rec_size:
                break
            try:
                data.append(struct.unpack(fmt, b))
            except struct.error:
                break
    return np.array(data)

# =====================================================
# Visualization helper
# =====================================================
def visualize_variant(name, arr, headers):
    print(f"📊 Visualizing {name} — {len(arr):,} records")

    df = pd.DataFrame(arr, columns=headers)
    display(df.head(25))  # top 25 rows preview

    # --- Timestamp deltas
    if "timestamp" in headers:
        ts = df["timestamp"].astype(np.float64)
        diffs = np.diff(ts)
        plt.figure(figsize=(6,3))
        plt.plot(diffs[:2000])
        plt.title(f"Timestamp Δ — {name}")
        plt.xlabel("Record #")
        plt.ylabel("Δ timestamp")
        plt.grid(True)
        plt.show()

    # --- Coordinate histograms
    cols = [c for c in headers if c.startswith(("x","y","z"))][:3]
    if cols:
        fig, axes = plt.subplots(1, len(cols), figsize=(12,3))
        if len(cols) == 1: axes = [axes]
        for ax, col in zip(axes, cols):
            data = df[col].astype(float)
            data = data[np.isfinite(data)]
            lo, hi = np.percentile(data, [0.1, 99.9])
            data = np.clip(data, lo, hi)
            ax.hist(data, bins=80, color="steelblue", alpha=0.7)
            ax.set_title(col)
        fig.suptitle(f"Coordinate Distributions — {name}")
        plt.show()

    # --- 3D scatter (if possible)
    if all(c in df.columns for c in ["x_i32","y_i32","z_i32"]) or \
       all(c in df.columns for c in ["x_f","y_f","z_f"]) or \
       all(c in df.columns for c in ["x_u32","y_u32","z_u32"]):
        # find matching coordinate columns dynamically
        x_col = [c for c in df.columns if c.startswith("x")][0]
        y_col = [c for c in df.columns if c.startswith("y")][0]
        z_col = [c for c in df.columns if c.startswith("z")][0]
        x, y, z = df[x_col].astype(float), df[y_col].astype(float), df[z_col].astype(float)
        mask = np.isfinite(x) & np.isfinite(y) & np.isfinite(z)
        x, y, z = x[mask], y[mask], z[mask]
        step = max(1, len(x)//20000)
        fig = plt.figure(figsize=(6,5))
        ax = fig.add_subplot(111, projection="3d")
        ax.scatter(x[::step], y[::step], z[::step], s=1, alpha=0.6)
        ax.set_xlabel(x_col); ax.set_ylabel(y_col); ax.set_zlabel(z_col)
        plt.title(f"3D Scatter — {name}")
        plt.show()

# =====================================================
# Run experiments
# =====================================================
for name, (fmt, headers) in candidates.items():
    print(f"\n🔹 Trying {name}: {fmt}")
    arr = decode_file(fmt)
    if len(arr) == 0:
        print(f"⚠️ {name}: no records decoded.")
        continue
    visualize_variant(name, arr, headers)

print("\n🎯 Done — all candidate decodings visualized.")
